In [0]:
# import helpermethods
import tensorflow as tf
import numpy as np
import sys
import os
import pandas as pd
sys.path.insert(0, '../../')

#Provide the GPU number to be used
os.environ['CUDA_VISIBLE_DEVICES'] ='-1'

#Bonsai imports
# from edgeml.trainer.bonsaiTrainer import BonsaiTrainer
# from edgeml.graph.bonsai import Bonsai

# Fixing seeds for reproducibility
tf.set_random_seed(42)
np.random.seed(42)

# Helper methods

In [0]:
import argparse
import datetime
import os
import numpy as np


def checkIntPos(value):
    ivalue = int(value)
    if ivalue <= 0:
        raise argparse.ArgumentTypeError(
            "%s is an invalid positive int value" % value)
    return ivalue


def checkIntNneg(value):
    ivalue = int(value)
    if ivalue < 0:
        raise argparse.ArgumentTypeError(
            "%s is an invalid non-neg int value" % value)
    return ivalue


def checkFloatNneg(value):
    fvalue = float(value)
    if fvalue < 0:
        raise argparse.ArgumentTypeError(
            "%s is an invalid non-neg float value" % value)
    return fvalue


def checkFloatPos(value):
    fvalue = float(value)
    if fvalue <= 0:
        raise argparse.ArgumentTypeError(
            "%s is an invalid positive float value" % value)
    return fvalue


def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')


def getArgs():
    '''
    Function to parse arguments for Bonsai Algorithm
    '''
    parser = argparse.ArgumentParser(
        description='HyperParams for Bonsai Algorithm')
    parser.add_argument('-dir', '--data-dir', required=True,
                        help='Data directory containing' +
                        'train.npy and test.npy')

    parser.add_argument('-d', '--depth', type=checkIntNneg, default=2,
                        help='Depth of Bonsai Tree ' +
                        '(default: 2 try: [0, 1, 3])')
    parser.add_argument('-p', '--proj-dim', type=checkIntPos, default=10,
                        help='Projection Dimension ' +
                        '(default: 20 try: [5, 10, 30])')
    parser.add_argument('-s', '--sigma', type=float, default=1.0,
                        help='Parameter for sigmoid sharpness ' +
                        '(default: 1.0 try: [3.0, 0.05, 0.1]')
    parser.add_argument('-e', '--epochs', type=checkIntPos, default=42,
                        help='Total Epochs (default: 42 try:[100, 150, 60])')
    parser.add_argument('-b', '--batch-size', type=checkIntPos,
                        help='Batch Size to be used ' +
                        '(default: max(100, sqrt(train_samples)))')
    parser.add_argument('-lr', '--learning-rate', type=checkFloatPos,
                        default=0.01, help='Initial Learning rate for ' +
                        'Adam Optimizer (default: 0.01)')

    parser.add_argument('-rW', type=float, default=0.0001,
                        help='Regularizer for predictor parameter W  ' +
                        '(default: 0.0001 try: [0.01, 0.001, 0.00001])')
    parser.add_argument('-rV', type=float, default=0.0001,
                        help='Regularizer for predictor parameter V  ' +
                        '(default: 0.0001 try: [0.01, 0.001, 0.00001])')
    parser.add_argument('-rT', type=float, default=0.0001,
                        help='Regularizer for branching parameter Theta  ' +
                        '(default: 0.0001 try: [0.01, 0.001, 0.00001])')
    parser.add_argument('-rZ', type=float, default=0.00001,
                        help='Regularizer for projection parameter Z  ' +
                        '(default: 0.00001 try: [0.001, 0.0001, 0.000001])')

    parser.add_argument('-sW', type=checkFloatPos,
                        help='Sparsity for predictor parameter W  ' +
                        '(default: For Binary classification 1.0 else 0.2 ' +
                        'try: [0.1, 0.3, 0.5])')
    parser.add_argument('-sV', type=checkFloatPos,
                        help='Sparsity for predictor parameter V  ' +
                        '(default: For Binary classification 1.0 else 0.2 ' +
                        'try: [0.1, 0.3, 0.5])')
    parser.add_argument('-sT', type=checkFloatPos,
                        help='Sparsity for branching parameter Theta  ' +
                        '(default: For Binary classification 1.0 else 0.2 ' +
                        'try: [0.1, 0.3, 0.5])')
    parser.add_argument('-sZ', type=checkFloatPos, default=0.2,
                        help='Sparsity for projection parameter Z  ' +
                        '(default: 0.2 try: [0.1, 0.3, 0.5])')
    parser.add_argument('-oF', '--output-file', default=None,
                        help='Output file for dumping the program output, ' +
                        '(default: stdout)')

    parser.add_argument('-regression', type=str2bool, default=False,
                        help='boolean argument which controls whether to perform ' +
                        'regression or classification.' +
                        'default : False (Classification) values: [True, False]')

    return parser.parse_args()


def getQuantArgs():
    '''
    Function to parse arguments for Model Quantisation
    '''
    parser = argparse.ArgumentParser(
        description='Arguments for quantizing Fast models. ' +
        'Works only for piece-wise linear non-linearities, ' +
        'like relu, quantTanh, quantSigm (check rnn.py for the definitions)')
    parser.add_argument('-dir', '--model-dir', required=True,
                        help='model directory containing' +
                        '*.npy weight files dumped from the trained model')
    parser.add_argument('-m', '--max-val', type=checkIntNneg, default=127,
                        help='this represents the maximum possible value ' +
                        'in model, essentially the byte complexity, ' +
                        '127=> 1 byte is default')

    return parser.parse_args()


def createTimeStampDir(dataDir):
    '''
    Creates a Directory with timestamp as it's name
    '''
    if os.path.isdir(dataDir + '/TFBonsaiResults') is False:
        try:
            os.mkdir(dataDir + '/TFBonsaiResults')
        except OSError:
            print("Creation of the directory %s failed" %
                  dataDir + '/TFBonsaiResults')

    currDir = 'TFBonsaiResults/' + datetime.datetime.now().strftime("%H_%M_%S_%d_%m_%y")
    if os.path.isdir(dataDir + '/' + currDir) is False:
        try:
            os.mkdir(dataDir + '/' + currDir)
        except OSError:
            print("Creation of the directory %s failed" %
                  dataDir + '/' + currDir)
        else:
            return (dataDir + '/' + currDir)
    return None


def preProcessData(dataDir, w,isRegression=False):
    '''
    Function to pre-process input data
    Expects a .npy file of form [lbl feats] for each datapoint
    Outputs a train and test set datapoints appended with 1 for Bias induction
    dataDimension, numClasses are inferred directly
    '''
    train = np.load(dataDir + '/train_'+str(w)+'.npy')
    test = np.load(dataDir + '/test_'+str(w)+'.npy')

    dataDimension = int(train.shape[1]) - 1

    Xtrain = train[:, 1:dataDimension + 1]
    Ytrain_ = train[:, 0]

    Xtest = test[:, 1:dataDimension + 1]
    Ytest_ = test[:, 0]

    # Mean Var Normalisation
    mean = np.mean(Xtrain, 0)
    std = np.std(Xtrain, 0)
    std[std[:] < 0.000001] = 1
    Xtrain = (Xtrain - mean) / std
    Xtest = (Xtest - mean) / std
    # End Mean Var normalisation

    # Classification.
    if (isRegression == False):
        numClasses = max(Ytrain_) - min(Ytrain_) + 1
        numClasses = int(max(numClasses, max(Ytest_) - min(Ytest_) + 1))

        lab = Ytrain_.astype('uint8')
        lab = np.array(lab) - min(lab)

        lab_ = np.zeros((Xtrain.shape[0], numClasses))
        lab_[np.arange(Xtrain.shape[0]), lab] = 1
        if (numClasses == 2):
            Ytrain = np.reshape(lab, [-1, 1])
        else:
            Ytrain = lab_

        lab = Ytest_.astype('uint8')
        lab = np.array(lab) - min(lab)

        lab_ = np.zeros((Xtest.shape[0], numClasses))
        lab_[np.arange(Xtest.shape[0]), lab] = 1
        if (numClasses == 2):
            Ytest = np.reshape(lab, [-1, 1])
        else:
            Ytest = lab_

    elif (isRegression == True):
        # The number of classes is always 1, for regression.
        numClasses = 1
        Ytrain = Ytrain_
        Ytest = Ytest_

    trainBias = np.ones([Xtrain.shape[0], 1])
    Xtrain = np.append(Xtrain, trainBias, axis=1)
    testBias = np.ones([Xtest.shape[0], 1])
    Xtest = np.append(Xtest, testBias, axis=1)

    mean = np.append(mean, np.array([0]))
    std = np.append(std, np.array([1]))
    
    if (isRegression == False):
        return dataDimension + 1, numClasses, Xtrain, Ytrain, Xtest, Ytest, mean, std
    elif (isRegression == True):
        return dataDimension + 1, numClasses, Xtrain, Ytrain.reshape((-1, 1)), Xtest, Ytest.reshape((-1, 1)), mean, std


def dumpCommand(list, currDir):
    '''
    Dumps the current command to a file for further use
    '''
    commandFile = open(currDir + '/command.txt', 'w')
    command = "python"

    command = command + " " + ' '.join(list)
    commandFile.write(command)

    commandFile.flush()
    commandFile.close()


def saveMeanStd(mean, std, currDir):
    '''
    Function to save Mean and Std vectors
    '''
    np.save(currDir + '/mean.npy', mean)
    np.save(currDir + '/std.npy', std)

#utils

In [0]:
#utils
import scipy.cluster
import scipy.spatial
import os


def medianHeuristic(data, projectionDimension, numPrototypes, W_init=None):
    '''
    This method can be used to estimate gamma for ProtoNN. An approximation to
    median heuristic is used here.
    1. First the data is collapsed into the projectionDimension by W_init. If
    W_init is not provided, it is initialized from a random normal(0, 1). Hence
    data normalization is essential.
    2. Prototype are computed by running a  k-means clustering on the projected
    data.
    3. The median distance is then estimated by calculating median distance
    between prototypes and projected data points.

    data needs to be [-1, numFeats]
    If using this method to initialize gamma, please use the W and B as well.

    TODO: Return estimate of Z (prototype labels) based on cluster centroids
    andand labels

    TODO: Clustering fails due to singularity error if projecting upwards

    W [dxd_cap]
    B [d_cap, m]
    returns gamma, W, B
    '''
    assert data.ndim == 2
    X = data
    featDim = data.shape[1]
    if projectionDimension > featDim:
        print("Warning: Projection dimension > feature dimension. Gamma")
        print("\t estimation due to median heuristic could fail.")
        print("\tTo retain the projection dataDimension, provide")
        print("\ta value for gamma.")

    if W_init is None:
        W_init = np.random.normal(size=[featDim, projectionDimension])
    W = W_init
    XW = np.matmul(X, W)
    assert XW.shape[1] == projectionDimension
    assert XW.shape[0] == len(X)
    # Requires [N x d_cap] data matrix of N observations of d_cap-dimension and
    # the number of centroids m. Returns, [n x d_cap] centroids and
    # elementwise center information.
    B, centers = scipy.cluster.vq.kmeans2(XW, numPrototypes)
    # Requires two matrices. Number of observations x dimension of observation
    # space. Distances[i,j] is the distance between XW[i] and B[j]
    distances = scipy.spatial.distance.cdist(XW, B, metric='euclidean')
    distances = np.reshape(distances, [-1])
    gamma = np.median(distances)
    gamma = 1 / (2.5 * gamma)
    return gamma.astype('float32'), W.astype('float32'), B.T.astype('float32')


def multiClassHingeLoss(logits, label, batch_th):
    '''
    MultiClassHingeLoss to match C++ Version - No TF internal version
    '''
    flatLogits = tf.reshape(logits, [-1, ])
    label_ = tf.argmax(label, 1)

    correctId = tf.range(0, batch_th) * label.shape[1] + label_
    correctLogit = tf.gather(flatLogits, correctId)

    maxLabel = tf.argmax(logits, 1)
    top2, _ = tf.nn.top_k(logits, k=2, sorted=True)

    wrongMaxLogit = tf.where(
        tf.equal(maxLabel, label_), top2[:, 1], top2[:, 0])

    return tf.reduce_mean(tf.nn.relu(1. + wrongMaxLogit - correctLogit))


def crossEntropyLoss(logits, label):
    '''
    Cross Entropy loss for MultiClass case in joint training for
    faster convergence
    '''
    return tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                   labels=tf.stop_gradient(label)))


def mean_absolute_error(logits, label):
    '''
    Function to compute the mean absolute error.
    '''
    return tf.reduce_mean(tf.abs(tf.subtract(logits, label)))


def hardThreshold(A, s):
    '''
    Hard thresholding function on Tensor A with sparsity s
    '''
    A_ = np.copy(A)
    A_ = A_.ravel()
    if len(A_) > 0:
        th = np.percentile(np.abs(A_), (1 - s) * 100.0, interpolation='higher')
        A_[np.abs(A_) < th] = 0.0
    A_ = A_.reshape(A.shape)
    return A_


def copySupport(src, dest):
    '''
    copy support of src tensor to dest tensor
    '''
    support = np.nonzero(src)
    dest_ = dest
    dest = np.zeros(dest_.shape)
    dest[support] = dest_[support]
    return dest


def countnnZ(A, s, bytesPerVar=4):
    '''
    Returns # of non-zeros and representative size of the tensor
    Uses dense for s >= 0.5 - 4 byte
    Else uses sparse - 8 byte
    '''
    params = 1
    hasSparse = False
    for i in range(0, len(A.shape)):
        params *= int(A.shape[i])
    if s < 0.5:
        nnZ = np.ceil(params * s)
        hasSparse = True
        return nnZ, nnZ * 2 * bytesPerVar, hasSparse
    else:
        nnZ = params
        return nnZ, nnZ * bytesPerVar, hasSparse


def getConfusionMatrix(predicted, target, numClasses):
    '''
    Returns a confusion matrix for a multiclass classification
    problem. `predicted` is a 1-D array of integers representing
    the predicted classes and `target` is the target classes.

    confusion[i][j]: Number of elements of class j
        predicted as class i
    Labels are assumed to be in range(0, numClasses)
    Use`printFormattedConfusionMatrix` to echo the confusion matrix
    in a user friendly form.
    '''
    assert(predicted.ndim == 1)
    assert(target.ndim == 1)
    arr = np.zeros([numClasses, numClasses])

    for i in range(len(predicted)):
        arr[predicted[i]][target[i]] += 1
    return arr


def printFormattedConfusionMatrix(matrix):
    '''
    Given a 2D confusion matrix, prints it in a human readable way.
    The confusion matrix is expected to be a 2D numpy array with
    square dimensions
    '''
    assert(matrix.ndim == 2)
    assert(matrix.shape[0] == matrix.shape[1])
    RECALL = 'Recall'
    PRECISION = 'PRECISION'
    print("|%s|" % ('True->'), end='')
    for i in range(matrix.shape[0]):
        print("%7d|" % i, end='')
    print("%s|" % 'Precision')

    print("|%s|" % ('-' * len(RECALL)), end='')
    for i in range(matrix.shape[0]):
        print("%s|" % ('-' * 7), end='')
    print("%s|" % ('-' * len(PRECISION)))

    precisionlist = np.sum(matrix, axis=1)
    recalllist = np.sum(matrix, axis=0)
    precisionlist = [matrix[i][i] / x if x !=
                     0 else -1 for i, x in enumerate(precisionlist)]
    recalllist = [matrix[i][i] / x if x !=
                  0 else -1 for i, x in enumerate(recalllist)]
    for i in range(matrix.shape[0]):
        # len recall = 6
        print("|%6d|" % (i), end='')
        for j in range(matrix.shape[0]):
            print("%7d|" % (matrix[i][j]), end='')
        print("%s" % (" " * (len(PRECISION) - 7)), end='')
        if precisionlist[i] != -1:
            print("%1.5f|" % precisionlist[i])
        else:
            print("%7s|" % "nan")

    print("|%s|" % ('-' * len(RECALL)), end='')
    for i in range(matrix.shape[0]):
        print("%s|" % ('-' * 7), end='')
    print("%s|" % ('-' * len(PRECISION)))
    print("|%s|" % ('Recall'), end='')

    for i in range(matrix.shape[0]):
        if recalllist[i] != -1:
            print("%1.5f|" % (recalllist[i]), end='')
        else:
            print("%7s|" % "nan", end='')

    print('%s|' % (' ' * len(PRECISION)))


def getPrecisionRecall(cmatrix, label=1):
    trueP = cmatrix[label][label]
    denom = np.sum(cmatrix, axis=0)[label]
    if denom == 0:
        denom = 1
    recall = trueP / denom
    denom = np.sum(cmatrix, axis=1)[label]
    if denom == 0:
        denom = 1
    precision = trueP / denom
    return precision, recall


def getMacroPrecisionRecall(cmatrix):
    # TP + FP
    precisionlist = np.sum(cmatrix, axis=1)
    # TP + FN
    recalllist = np.sum(cmatrix, axis=0)
    precisionlist__ = [cmatrix[i][i] / x if x !=
                       0 else 0 for i, x in enumerate(precisionlist)]
    recalllist__ = [cmatrix[i][i] / x if x !=
                    0 else 0 for i, x in enumerate(recalllist)]
    precision = np.sum(precisionlist__)
    precision /= len(precisionlist__)
    recall = np.sum(recalllist__)
    recall /= len(recalllist__)
    return precision, recall


def getMicroPrecisionRecall(cmatrix):
    # TP + FP
    precisionlist = np.sum(cmatrix, axis=1)
    # TP + FN
    recalllist = np.sum(cmatrix, axis=0)
    num = 0.0
    for i in range(len(cmatrix)):
        num += cmatrix[i][i]

    precision = num / np.sum(precisionlist)
    recall = num / np.sum(recalllist)
    return precision, recall


def getMacroMicroFScore(cmatrix):
    '''
    Returns macro and micro f-scores.
    Refer: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.104.8244&rep=rep1&type=pdf
    '''
    precisionlist = np.sum(cmatrix, axis=1)
    recalllist = np.sum(cmatrix, axis=0)
    precisionlist__ = [cmatrix[i][i] / x if x !=
                       0 else 0 for i, x in enumerate(precisionlist)]
    recalllist__ = [cmatrix[i][i] / x if x !=
                    0 else 0 for i, x in enumerate(recalllist)]
    macro = 0.0
    for i in range(len(precisionlist)):
        denom = precisionlist__[i] + recalllist__[i]
        numer = precisionlist__[i] * recalllist__[i] * 2
        if denom == 0:
            denom = 1
        macro += numer / denom
    macro /= len(precisionlist)

    num = 0.0
    for i in range(len(precisionlist)):
        num += cmatrix[i][i]

    denom1 = np.sum(precisionlist)
    denom2 = np.sum(recalllist)
    pi = num / denom1
    rho = num / denom2
    denom = pi + rho
    if denom == 0:
        denom = 1
    micro = 2 * pi * rho / denom
    return macro, micro


class GraphManager:
    '''
    Manages saving and restoring graphs. Designed to be used with EMI-RNN
    though is general enough to be useful otherwise as well.
    '''

    def __init__(self):
        pass

    def checkpointModel(self, saver, sess, modelPrefix,
                        globalStep=1000, redirFile=None):
        saver.save(sess, modelPrefix, global_step=globalStep)
        print('Model saved to %s, global_step %d' % (modelPrefix, globalStep),
              file=redirFile)

    def loadCheckpoint(self, sess, modelPrefix, globalStep,
                       redirFile=None):
        metaname = modelPrefix + '-%d.meta' % globalStep
        basename = os.path.basename(metaname)
        fileList = os.listdir(os.path.dirname(modelPrefix))
        fileList = [x for x in fileList if x.startswith(basename)]
        assert len(fileList) > 0, 'Checkpoint file not found'
        msg = 'Too many or too few checkpoint files for globalStep: %d' % globalStep
        assert len(fileList) is 1, msg
        chkpt = basename + '/' + fileList[0]
        saver = tf.train.import_meta_graph(metaname)
        metaname = metaname[:-5]
        saver.restore(sess, metaname)
        graph = tf.get_default_graph()
        return graph

# Bonsai.py

In [0]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT license.
import warnings


class Bonsai:
    def __init__(self, numClasses, dataDimension, projectionDimension,
                 treeDepth, sigma,
                 isRegression=False, W=None, T=None, V=None, Z=None):
        '''
        Expected Dimensions:

        Bonsai Params // Optional
        W [numClasses*totalNodes, projectionDimension]
        V [numClasses*totalNodes, projectionDimension]
        Z [projectionDimension, dataDimension + 1]
        T [internalNodes, projectionDimension]

        internalNodes = 2**treeDepth - 1
        totalNodes = 2*internalNodes + 1

        sigma - tanh non-linearity
        sigmaI - Indicator function for node probabilities
        sigmaI - has to be set to infinity(1e9 for practicality)
        while doing testing/inference
        numClasses will be reset to 1 in binary case
        '''
        self.dataDimension = dataDimension
        self.projectionDimension = projectionDimension
        self.isRegression = isRegression

        if ((self.isRegression == True) & (numClasses != 1)):
            warnings.warn("Number of classes cannot be greater than 1 for regression")
            self.numClasses = 1

        if numClasses == 2:
            self.numClasses = 1
        else:
            self.numClasses = numClasses

        self.treeDepth = treeDepth
        self.sigma = sigma

        self.internalNodes = 2**self.treeDepth - 1
        self.totalNodes = 2 * self.internalNodes + 1

        self.W = self.initW(W)
        self.V = self.initV(V)
        self.T = self.initT(T)
        self.Z = self.initZ(Z)

        self.assertInit()

        self.score = None
        self.X_ = None
        self.prediction = None

    def initZ(self, Z):
        if Z is None:
            Z = tf.random_normal(
                [self.projectionDimension, self.dataDimension])
        Z = tf.Variable(Z, name='Z', dtype=tf.float32)
        return Z

    def initW(self, W):
        if W is None:
            W = tf.random_normal(
                [self.numClasses * self.totalNodes, self.projectionDimension])
        W = tf.Variable(W, name='W', dtype=tf.float32)
        return W

    def initV(self, V):
        if V is None:
            V = tf.random_normal(
                [self.numClasses * self.totalNodes, self.projectionDimension])
        V = tf.Variable(V, name='V', dtype=tf.float32)
        return V

    def initT(self, T):
        if T is None:
            T = tf.random_normal(
                [self.internalNodes, self.projectionDimension])
        T = tf.Variable(T, name='T', dtype=tf.float32)
        return T

    def __call__(self, X, sigmaI):
        '''
        Function to build the Bonsai Tree graph
        Expected Dimensions

        X is [_, self.dataDimension]
        '''
        errmsg = "Dimension Mismatch, X is [_, self.dataDimension]"
        assert (len(X.shape) == 2 and int(
            X.shape[1]) == self.dataDimension), errmsg
        if self.score is not None:
            return self.score, self.X_

        X_ = tf.divide(tf.matmul(self.Z, X, transpose_b=True),
                       self.projectionDimension)

        W_ = self.W[0:(self.numClasses)]
        V_ = self.V[0:(self.numClasses)]

        self.__nodeProb = []
        self.__nodeProb.append(1)

        score_ = self.__nodeProb[0] * tf.multiply(
            tf.matmul(W_, X_), tf.tanh(self.sigma * tf.matmul(V_, X_)))
        for i in range(1, self.totalNodes):
            W_ = self.W[i * self.numClasses:((i + 1) * self.numClasses)]
            V_ = self.V[i * self.numClasses:((i + 1) * self.numClasses)]

            T_ = tf.reshape(self.T[int(np.ceil(i / 2.0) - 1.0)],
                            [-1, self.projectionDimension])
            prob = (1 + ((-1)**(i + 1)) *
                    tf.tanh(tf.multiply(sigmaI, tf.matmul(T_, X_))))

            prob = tf.divide(prob, 2.0)
            prob = self.__nodeProb[int(np.ceil(i / 2.0) - 1.0)] * prob
            self.__nodeProb.append(prob)
            score_ += self.__nodeProb[i] * tf.multiply(
                tf.matmul(W_, X_), tf.tanh(self.sigma * tf.matmul(V_, X_)))

        self.score = score_
        self.X_ = X_
        return self.score, self.X_

    def getPrediction(self):
        '''
        Takes in a score tensor and outputs a integer class for each data point
        '''

        # Classification.
        if (self.isRegression == False):
            if self.prediction is not None:
                return self.prediction

            if self.numClasses > 2:
                self.prediction = tf.argmax(tf.transpose(self.score), 1)
            else:
                self.prediction = tf.argmax(
                    tf.concat([tf.transpose(self.score),
                               0 * tf.transpose(self.score)], 1), 1)
        # Regression.
        elif (self.isRegression == True):
            # For regression , scores are the actual predictions, just return them.
            self.prediction = self.score    
        return self.prediction

    def assertInit(self):
        errmsg = "Number of Classes for regression can only be 1."
        if (self.isRegression == True):
            assert (self.numClasses == 1), errmsg
        errRank = "All Parameters must has only two dimensions shape = [a, b]"
        assert len(self.W.shape) == len(self.Z.shape), errRank
        assert len(self.W.shape) == len(self.T.shape), errRank
        assert len(self.W.shape) == 2, errRank
        msg = "W and V should be of same Dimensions"
        assert self.W.shape == self.V.shape, msg
        errW = "W and V are [numClasses*totalNodes, projectionDimension]"
        assert self.W.shape[0] == self.numClasses * self.totalNodes, errW
        assert self.W.shape[1] == self.projectionDimension, errW
        errZ = "Z is [projectionDimension, dataDimension]"
        assert self.Z.shape[0] == self.projectionDimension, errZ
        assert self.Z.shape[1] == self.dataDimension, errZ
        errT = "T is [internalNodes, projectionDimension]"
        assert self.T.shape[0] == self.internalNodes, errT
        assert self.T.shape[1] == self.projectionDimension, errT
        assert int(self.numClasses) > 0, "numClasses should be > 1"
        msg = "# of features in data should be > 0"
        assert int(self.dataDimension) > 0, msg
        msg = "Projection should be  > 0 dims"
        assert int(self.projectionDimension) > 0, msg
        msg = "treeDepth should be >= 0"
        assert int(self.treeDepth) >= 0, msg


# bonsaiTrainer.py

In [0]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT license.

from __future__ import print_function
# import edgeml.utils as utils


class BonsaiTrainer:
    def __init__(self, bonsaiObj, lW, lT, lV, lZ, sW, sT, sV, sZ,
                 learningRate, X, Y, useMCHLoss=False, outFile=None, regLoss='huber'):
        '''
        bonsaiObj - Initialised Bonsai Object and Graph
        lW, lT, lV and lZ are regularisers to Bonsai Params
        sW, sT, sV and sZ are sparsity factors to Bonsai Params
        learningRate - learningRate fro optimizer
        X is the Data Placeholder - Dims [_, dataDimension]
        Y - Label placeholder for loss computation
        useMCHLoss - For choice between HingeLoss vs CrossEntropy
        useMCHLoss - True - MultiClass - multiClassHingeLoss
        useMCHLoss - False - MultiClass - crossEntropyLoss
        '''
        self.testAcc1 = 0
        self.testLoss1 = 0
        self.regTestLoss1= 0
        self.pred1 = 0
        
        self.bonsaiObj = bonsaiObj
        self.regressionLoss = regLoss

        self.lW = lW
        self.lV = lV
        self.lT = lT
        self.lZ = lZ

        self.sW = sW
        self.sV = sV
        self.sT = sT
        self.sZ = sZ

        self.Y = Y
        self.X = X

        self.useMCHLoss = useMCHLoss

        if outFile is not None:
            print("Outfile : ", outFile)
            self.outFile = open(outFile, 'w')
        else:
            self.outFile = sys.stdout

        self.learningRate = learningRate

        self.assertInit()

        self.sigmaI = tf.placeholder(tf.float32, name='sigmaI')

        self.score, self.X_ = self.bonsaiObj(self.X, self.sigmaI)

        self.loss, self.marginLoss, self.regLoss = self.lossGraph()

        self.trainStep = self.trainGraph()
        '''
        self.accuracy -> 'MAE' for Regression.
        self.accuracy -> 'Accuracy' for Classification.
        '''
        self.accuracy = self.accuracyGraph()
        self.prediction = self.bonsaiObj.getPrediction()

        if self.sW > 0.99 and self.sV > 0.99 and self.sZ > 0.99 and self.sT > 0.99:
            self.isDenseTraining = True
        else:
            self.isDenseTraining = False

        self.hardThrsd()
        self.sparseTraining()

    def lossGraph(self):
        '''
        Loss Graph for given Bonsai Obj
        '''
        self.regLoss = 0.5 * (self.lZ * tf.square(tf.norm(self.bonsaiObj.Z)) +
                              self.lW * tf.square(tf.norm(self.bonsaiObj.W)) +
                              self.lV * tf.square(tf.norm(self.bonsaiObj.V)) +
                              self.lT * tf.square(tf.norm(self.bonsaiObj.T)))

        # Loss functions for classification.
        if (self.bonsaiObj.isRegression == False):
            if (self.bonsaiObj.numClasses > 2):
                if self.useMCHLoss is True:
                    self.batch_th = tf.placeholder(tf.int64, name='batch_th')
                    self.marginLoss = utils.multiClassHingeLoss(
                        tf.transpose(self.score), self.Y,
                        self.batch_th)
                else:
                    self.marginLoss = utils.crossEntropyLoss(
                        tf.transpose(self.score), self.Y)
                self.loss = self.marginLoss + self.regLoss
            else:
                self.marginLoss = tf.reduce_mean(tf.nn.relu(
                    1.0 - (2 * self.Y - 1) * tf.transpose(self.score)))
                self.loss = self.marginLoss + self.regLoss

        # Loss functions for regression.
        elif (self.bonsaiObj.isRegression == True):
            if(self.regressionLoss == 'huber'):
                # Use of Huber Loss , because it is more robust to outliers.
                self.marginLoss = tf.losses.huber_loss(self.Y, tf.transpose(self.score))
                self.loss = self.marginLoss + self.regLoss
            elif (self.regressionLoss == 'l2'):
                # L2 loss function.
                self.marginLoss = tf.nn.l2_loss(self.Y - tf.transpose(self.score))
                self.loss = self.marginLoss + self.regLoss

        return self.loss, self.marginLoss, self.regLoss

    def trainGraph(self):
        '''
        Train Graph for the loss generated by Bonsai
        '''
        self.bonsaiObj.TrainStep = tf.train.AdamOptimizer(
            self.learningRate).minimize(self.loss)

        return self.bonsaiObj.TrainStep

    def accuracyGraph(self):
        '''
        Accuracy Graph to evaluate accuracy when needed
        '''
        if(self.bonsaiObj.isRegression == False):
            if (self.bonsaiObj.numClasses > 2):
                correctPrediction = tf.equal(
                    tf.argmax(tf.transpose(self.score), 1), tf.argmax(self.Y, 1))
                self.accuracy = tf.reduce_mean(tf.cast(correctPrediction, tf.float32))
            else:
                y_ = self.Y * 2 - 1
                correctPrediction = tf.multiply(tf.transpose(self.score), y_)
                correctPrediction = tf.nn.relu(correctPrediction)
                correctPrediction = tf.ceil(tf.tanh(correctPrediction))
                self.accuracy = tf.reduce_mean(tf.cast(correctPrediction, tf.float32))

        elif (self.bonsaiObj.isRegression == True):
            # Accuracy for regression , in terms of mean absolute error.
            self.accuracy = mean_absolute_error(tf.reshape(
                self.score, [-1, 1]), tf.reshape(self.Y, [-1, 1]))
        return self.accuracy

    def hardThrsd(self):
        '''
        Set up for hard Thresholding Functionality
        '''
        self.__Wth = tf.placeholder(tf.float32, name='Wth')
        self.__Vth = tf.placeholder(tf.float32, name='Vth')
        self.__Zth = tf.placeholder(tf.float32, name='Zth')
        self.__Tth = tf.placeholder(tf.float32, name='Tth')

        self.__Woph = self.bonsaiObj.W.assign(self.__Wth)
        self.__Voph = self.bonsaiObj.V.assign(self.__Vth)
        self.__Toph = self.bonsaiObj.T.assign(self.__Tth)
        self.__Zoph = self.bonsaiObj.Z.assign(self.__Zth)

        self.hardThresholdGroup = tf.group(
            self.__Woph, self.__Voph, self.__Toph, self.__Zoph)

    def sparseTraining(self):
        '''
        Set up for Sparse Retraining Functionality
        '''
        self.__Wops = self.bonsaiObj.W.assign(self.__Wth)
        self.__Vops = self.bonsaiObj.V.assign(self.__Vth)
        self.__Zops = self.bonsaiObj.Z.assign(self.__Zth)
        self.__Tops = self.bonsaiObj.T.assign(self.__Tth)

        self.sparseRetrainGroup = tf.group(
            self.__Wops, self.__Vops, self.__Tops, self.__Zops)

    def runHardThrsd(self, sess):
        '''
        Function to run the IHT routine on Bonsai Obj
        '''
        currW = self.bonsaiObj.W.eval()
        currV = self.bonsaiObj.V.eval()
        currZ = self.bonsaiObj.Z.eval()
        currT = self.bonsaiObj.T.eval()

        self.__thrsdW = hardThreshold(currW, self.sW)
        self.__thrsdV = hardThreshold(currV, self.sV)
        self.__thrsdZ = hardThreshold(currZ, self.sZ)
        self.__thrsdT = hardThreshold(currT, self.sT)

        fd_thrsd = {self.__Wth: self.__thrsdW, self.__Vth: self.__thrsdV,
                    self.__Zth: self.__thrsdZ, self.__Tth: self.__thrsdT}
        sess.run(self.hardThresholdGroup, feed_dict=fd_thrsd)

    def runSparseTraining(self, sess):
        '''
        Function to run the Sparse Retraining routine on Bonsai Obj
        '''
        currW = self.bonsaiObj.W.eval()
        currV = self.bonsaiObj.V.eval()
        currZ = self.bonsaiObj.Z.eval()
        currT = self.bonsaiObj.T.eval()

        newW = copySupport(self.__thrsdW, currW)
        newV = copySupport(self.__thrsdV, currV)
        newZ = copySupport(self.__thrsdZ, currZ)
        newT = copySupport(self.__thrsdT, currT)

        fd_st = {self.__Wth: newW, self.__Vth: newV,
                 self.__Zth: newZ, self.__Tth: newT}
        sess.run(self.sparseRetrainGroup, feed_dict=fd_st)

    def assertInit(self):
        err = "sparsity must be between 0 and 1"
        assert self.sW >= 0 and self.sW <= 1, "W " + err
        assert self.sV >= 0 and self.sV <= 1, "V " + err
        assert self.sZ >= 0 and self.sZ <= 1, "Z " + err
        assert self.sT >= 0 and self.sT <= 1, "T " + err
        errMsg = "Dimension Mismatch, Y has to be [_, " + \
            str(self.bonsaiObj.numClasses) + "]"
        errCont = " numClasses are 1 in case of Binary case by design"
        assert (len(self.Y.shape) == 2 and
                self.Y.shape[1] == self.bonsaiObj.numClasses), errMsg + errCont

    def saveParams(self, currDir):
        '''
        Function to save Parameter matrices into a given folder
        '''
        paramDir = currDir + '/'
        np.save(paramDir + "W.npy", self.bonsaiObj.W.eval())
        np.save(paramDir + "V.npy", self.bonsaiObj.V.eval())
        np.save(paramDir + "T.npy", self.bonsaiObj.T.eval())
        np.save(paramDir + "Z.npy", self.bonsaiObj.Z.eval())
        hyperParamDict = {'dataDim': self.bonsaiObj.dataDimension,
                          'projDim': self.bonsaiObj.projectionDimension,
                          'numClasses': self.bonsaiObj.numClasses,
                          'depth': self.bonsaiObj.treeDepth,
                          'sigma': self.bonsaiObj.sigma}
        hyperParamFile = paramDir + 'hyperParam.npy'
        np.save(hyperParamFile, hyperParamDict)

    def loadModel(self, currDir):
        '''
        Load the Saved model and load it to the model using constructor
        Returns two dict one for params and other for hyperParams
        '''
        paramDir = currDir + '/'
        paramDict = {}
        paramDict['W'] = np.load(paramDir + "W.npy")
        paramDict['V'] = np.load(paramDir + "V.npy")
        paramDict['T'] = np.load(paramDir + "T.npy")
        paramDict['Z'] = np.load(paramDir + "Z.npy")
        hyperParamDict = np.load(paramDir + "hyperParam.npy").item()
        return paramDict, hyperParamDict

    # Function to get aimed model size
    def getModelSize(self):
        '''
        Function to get aimed model size
        '''
        nnzZ, sizeZ, sparseZ = countnnZ(self.bonsaiObj.Z, self.sZ)
        nnzW, sizeW, sparseW = countnnZ(self.bonsaiObj.W, self.sW)
        nnzV, sizeV, sparseV = countnnZ(self.bonsaiObj.V, self.sV)
        nnzT, sizeT, sparseT = countnnZ(self.bonsaiObj.T, self.sT)

        totalnnZ = (nnzZ + nnzT + nnzV + nnzW)
        totalSize = (sizeZ + sizeW + sizeV + sizeT)
        hasSparse = (sparseW or sparseV or sparseT or sparseZ)
        return totalnnZ, totalSize, hasSparse
    
    
    def train(self, batchSize, totalEpochs, sess,
              Xtrain, Xtest, Ytrain, Ytest, dataDir, currDir):
        '''
        The Dense - IHT - Sparse Retrain Routine for Bonsai Training
        '''
        resultFile = open(dataDir + '/TFBonsaiResults.txt', 'a+')
        numIters = Xtrain.shape[0] / batchSize

        totalBatches = numIters * totalEpochs

        bonsaiObjSigmaI = 1

        counter = 0
        if self.bonsaiObj.numClasses > 2:
            trimlevel = 15
        else:
            trimlevel = 5
        ihtDone = 0
        if (self.bonsaiObj.isRegression == True):
            maxTestAcc = 100000007
        else:
            maxTestAcc = -10000
        if self.isDenseTraining is True:
            ihtDone = 1
            bonsaiObjSigmaI = 1
            itersInPhase = 0

        header = '*' * 20
        for i in range(totalEpochs):
            print("\nEpoch Number: " + str(i), file=self.outFile)

            '''
            trainAcc -> For Regression, it is 'Mean Absolute Error'.
            trainAcc -> For Classification, it is 'Accuracy'.
            '''
            trainAcc = 0.0
            trainLoss = 0.0

            numIters = int(numIters)
            for j in range(numIters):

                if counter == 0:
                    msg = " Dense Training Phase Started "
                    print("\n%s%s%s\n" %
                          (header, msg, header), file=self.outFile)

                # Updating the indicator sigma
                if ((counter == 0) or (counter == int(totalBatches / 3.0)) or
                        (counter == int(2 * totalBatches / 3.0))) and (self.isDenseTraining is False):
                    bonsaiObjSigmaI = 1
                    itersInPhase = 0

                elif (itersInPhase % 100 == 0):
                    indices = np.random.choice(Xtrain.shape[0], 100)
                    batchX = Xtrain[indices, :]
                    batchY = Ytrain[indices, :]
                    batchY = np.reshape(
                        batchY, [-1, self.bonsaiObj.numClasses])

                    _feed_dict = {self.X: batchX}
                    Xcapeval = self.X_.eval(feed_dict=_feed_dict)
                    Teval = self.bonsaiObj.T.eval()

                    sum_tr = 0.0
                    for k in range(0, self.bonsaiObj.internalNodes):
                        sum_tr += (np.sum(np.abs(np.dot(Teval[k], Xcapeval))))

                    if(self.bonsaiObj.internalNodes > 0):
                        sum_tr /= (100 * self.bonsaiObj.internalNodes)
                        sum_tr = 0.1 / sum_tr
                    else:
                        sum_tr = 0.1
                    sum_tr = min(
                        1000, sum_tr * (2**(float(itersInPhase) /
                                            (float(totalBatches) / 30.0))))

                    bonsaiObjSigmaI = sum_tr

                itersInPhase += 1
                batchX = Xtrain[j * batchSize:(j + 1) * batchSize]
                batchY = Ytrain[j * batchSize:(j + 1) * batchSize]
                batchY = np.reshape(
                    batchY, [-1, self.bonsaiObj.numClasses])

                if self.bonsaiObj.numClasses > 2:
                    if self.useMCHLoss is True:
                        _feed_dict = {self.X: batchX, self.Y: batchY,
                                      self.batch_th: batchY.shape[0],
                                      self.sigmaI: bonsaiObjSigmaI}
                    else:
                        _feed_dict = {self.X: batchX, self.Y: batchY,
                                      self.sigmaI: bonsaiObjSigmaI}
                else:
                    _feed_dict = {self.X: batchX, self.Y: batchY,
                                  self.sigmaI: bonsaiObjSigmaI}

                # Mini-batch training
                _, batchLoss, batchAcc = sess.run(
                    [self.trainStep, self.loss, self.accuracy],
                    feed_dict=_feed_dict)

                # Classification.
                if (self.bonsaiObj.isRegression == False):
                    trainAcc += batchAcc
                    trainLoss += batchLoss
                # Regression.
                else:
                    trainAcc += np.mean(batchAcc)
                    trainLoss += np.mean(batchLoss)

                # Training routine involving IHT and sparse retraining
                if (counter >= int(totalBatches / 3.0) and
                    (counter < int(2 * totalBatches / 3.0)) and
                    counter % trimlevel == 0 and
                        self.isDenseTraining is False):
                    self.runHardThrsd(sess)
                    if ihtDone == 0:
                        msg = " IHT Phase Started "
                        print("\n%s%s%s\n" %
                              (header, msg, header), file=self.outFile)
                    ihtDone = 1
                elif ((ihtDone == 1 and counter >= int(totalBatches / 3.0) and
                       (counter < int(2 * totalBatches / 3.0)) and
                       counter % trimlevel != 0 and
                       self.isDenseTraining is False) or
                        (counter >= int(2 * totalBatches / 3.0) and
                            self.isDenseTraining is False)):
                    self.runSparseTraining(sess)
                    if counter == int(2 * totalBatches / 3.0):
                        msg = " Sparse Retraining Phase Started "
                        print("\n%s%s%s\n" %
                              (header, msg, header), file=self.outFile)
                counter += 1
            try:
                if (self.bonsaiObj.isRegression == True):
                    print("\nRegression Train Loss: " + str(trainLoss / numIters) +
                          "\nTraining MAE (Regression): " + str(trainAcc / numIters),
                          file=self.outFile)
                else:
                    print("\nClassification Train Loss: " + str(trainLoss / numIters) +
                          "\nTraining accuracy (Classification): " + str(trainAcc / numIters),
                          file=self.outFile)
            except:
                continue

            oldSigmaI = bonsaiObjSigmaI
            bonsaiObjSigmaI = 1e9

            if self.bonsaiObj.numClasses > 2:
                if self.useMCHLoss is True:
                    _feed_dict = {self.X: Xtest, self.Y: Ytest,
                                  self.batch_th: Ytest.shape[0],
                                  self.sigmaI: bonsaiObjSigmaI}
                else:
                    _feed_dict = {self.X: Xtest, self.Y: Ytest,
                                  self.sigmaI: bonsaiObjSigmaI}
            else:
                _feed_dict = {self.X: Xtest, self.Y: Ytest,
                              self.sigmaI: bonsaiObjSigmaI}

            # This helps in direct testing instead of extracting the model out

            testAcc, testLoss, regTestLoss, pred = sess.run(
                [self.accuracy, self.loss, self.regLoss, self.prediction], feed_dict=_feed_dict)
            self.testAcc1, self.testLoss1, self.regTestLoss1, self.pred1 = sess.run(
                [self.accuracy, self.loss, self.regLoss, self.prediction], feed_dict=_feed_dict)

            if ihtDone == 0:
                if (self.bonsaiObj.isRegression == False):
                    maxTestAcc = -10000
                    maxTestAccEpoch = i
                elif (self.bonsaiObj.isRegression == True):
                    maxTestAcc = testAcc
                    maxTestAccEpoch = i

            else:
                if (self.bonsaiObj.isRegression == False):
                    if maxTestAcc <= testAcc:
                        maxTestAccEpoch = i
                        maxTestAcc = testAcc
                        self.saveParams(currDir)
                elif (self.bonsaiObj.isRegression == True):
                    print("Minimum Training MAE : ", np.mean(maxTestAcc))
                    if maxTestAcc >= testAcc:
                        # For regression , we're more interested in the minimum MAE.
                        maxTestAccEpoch = i
                        maxTestAcc = testAcc
                        self.saveParams(currDir)

            if (self.bonsaiObj.isRegression == True):
                print("Testing MAE %g" % np.mean(testAcc), file=self.outFile)
            else:
                print("Test accuracy %g" % np.mean(testAcc), file=self.outFile)
#                 print("Prediction %g" %pred ,file=self.outFile)

            if (self.bonsaiObj.isRegression == True):
                testAcc = np.mean(testAcc)
            else:
                testAcc = testAcc
                maxTestAcc = maxTestAcc

            print("MarginLoss + RegLoss: " + str(testLoss - regTestLoss) +
                  " + " + str(regTestLoss) + " = " + str(testLoss) + "\n",
                  file=self.outFile)
            self.outFile.flush()

            bonsaiObjSigmaI = oldSigmaI

        # sigmaI has to be set to infinity to ensure
        # only a single path is used in inference
        bonsaiObjSigmaI = 1e9
        print("\nNon-Zero : " + str(self.getModelSize()[0]) + " Model Size: " +
              str(float(self.getModelSize()[1]) / 1024.0) + " KB hasSparse: " +
              str(self.getModelSize()[2]) + "\n", file=self.outFile)

        if (self.bonsaiObj.isRegression == True):
            maxTestAcc = np.mean(maxTestAcc)

        if (self.bonsaiObj.isRegression == True):
            print("For Regression, Minimum MAE at compressed" +
                  " model size(including early stopping): " +
                  str(maxTestAcc) + " at Epoch: " +
                  str(maxTestAccEpoch + 1) + "\nFinal Test" +
                  " MAE: " + str(testAcc), file=self.outFile)

            resultFile.write("MinTestMAE: " + str(maxTestAcc) +
                             " at Epoch(totalEpochs): " +
                             str(maxTestAccEpoch + 1) +
                             "(" + str(totalEpochs) + ")" + " ModelSize: " +
                             str(float(self.getModelSize()[1]) / 1024.0) +
                             " KB hasSparse: " + str(self.getModelSize()[2]) +
                             " Param Directory: " +
                             str(os.path.abspath(currDir)) + "\n")

        elif (self.bonsaiObj.isRegression == False):
            print("For Classification, Maximum Test accuracy at compressed" +
                  " model size(including early stopping): " +
                  str(maxTestAcc) + " at Epoch: " +
                  str(maxTestAccEpoch + 1) + "\nFinal Test" +
                  " Accuracy: " + str(testAcc), file=self.outFile)

            resultFile.write("MaxTestAcc: " + str(maxTestAcc) +
                             " at Epoch(totalEpochs): " +
                             str(maxTestAccEpoch + 1) +
                             "(" + str(totalEpochs) + ")" + " ModelSize: " +
                             str(float(self.getModelSize()[1]) / 1024.0) +
                             " KB hasSparse: " + str(self.getModelSize()[2]) +
                             " Param Directory: " +
                             str(os.path.abspath(currDir)) + "\n")
        print("The Model Directory: " + currDir + "\n")

        resultFile.close()
        self.outFile.flush()

        if self.outFile is not sys.stdout:
            self.outFile.close()


# Obtain data

In [0]:

# os.chdir("/home/iot/Documents/dataset_fog_release/dataset/ML")
# x_train = pd.read_csv("final_x.csv")
# x_train.drop('Unnamed: 0',axis=1,inplace=True)
# y_list = pd.read_csv("final_y.csv")
# y_list.drop('Unnamed: 0',axis=1,inplace=True)
# y_list = y_list-1
# x =pd.concat([x_train,y_list],axis=1)
# x= x[['0','mean1', 'mean2', 'mean9', 'rms2', 'abovemn2', 'belowmn2', 'soc4',
#        'PSD2', 'FI1', 'FI2', 'FI3', 'FI5', 'FI9', 'FImag2']]


In [0]:
from google.colab import drive 
drive.mount ("/content/gdrive")

DATA_DIR = "/content/gdrive/My Drive/Recovery/dataset_fog_release/dataset/DATA_DIR"
windowLen = '2'
(dataDimension, numClasses, Xtrain, Ytrain, Xtest, Ytest, mean, std) = preProcessData(DATA_DIR,windowLen, isRegression=False)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Model Parameters

Note that Bonsai is designed for low-memory setting and the best results are obtained when operating in that setting. Use the sparsity, projection dimension and tree depth to vary the model size.

In [0]:
sigma = 1.0 #Sigmoid parameter for tanh
depth = 10 #Depth of Bonsai Tree
projectionDimension = 48 #Lower Dimensional space for Bonsai to work on

#Regularizers for Bonsai Parameters
regZ = 0.0001
regW = 0.001
regV = 0.001
regT = 0.001

totalEpochs = 100

learningRate = 0.01

outFile = None

#Sparsity for Bonsai Parameters. x => 100*x % are non-zeros
sparZ = 0.2
sparW = 0.3
sparV = 0.3
sparT = 0.62

batchSize = np.maximum(100, int(np.ceil(np.sqrt(Ytrain.shape[0]))))

useMCHLoss = True #only for Multiclass cases True: Multiclass-Hing Loss, False: Cross Entropy. 

#Bonsai uses one classier for Binary, thus this condition
if numClasses == 2:
    numClasses = 1

Placeholders for Data feeding during training and infernece

In [0]:
X = tf.placeholder("float32", [None, dataDimension])
Y = tf.placeholder("float32", [None, numClasses])

Creating a directory for current model in the datadirectory using timestamp

In [0]:
currDir = createTimeStampDir(DATA_DIR)
dumpCommand(sys.argv, currDir)

# Bonsai Graph Object

Instantiating the Bonsai Graph which will be used for training and inference.

In [0]:
bonsaiObj = Bonsai(numClasses, dataDimension, projectionDimension, depth, sigma)

# Bonsai Trainer Object

Instantiating the Bonsai Trainer which will be used for 3 phase training.

In [0]:
bonsaiTrainer = BonsaiTrainer(bonsaiObj, regW, regT, regV, regZ, sparW, sparT, sparV, sparZ,
                              learningRate, X, Y, useMCHLoss, outFile)

In [0]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# Bonsai Training Routine

The method to to run the 3 phase training, followed by giving out the best early stopping model, accuracy along with saving of the parameters.

In [0]:
bonsaiTrainer.train(batchSize, totalEpochs, sess,
                    Xtrain, Xtest, Ytrain, Ytest, DATA_DIR, currDir)


Epoch Number: 0

******************** Dense Training Phase Started ********************



In [0]:
pred = bonsaiTrainer.pred1
y_pred=[]
for i in pred:
  if i == 0:
    y_pred.append(1)
  else:
    y_pred.append(0)

In [0]:
from sklearn.metrics import confusion_matrix,classification_report
# Ytest = np.argmax(Ytest,axis=1)
print (confusion_matrix(Ytest,y_pred))
print (classification_report(Ytest,y_pred,digits=5))